In [1]:
import os
import pandas as pd
import numpy as np
import math
from matplotlib.pyplot import *
import matplotlib.pyplot as plt
from matplotlib import animation
from matplotlib import cm
from parse import *
import seaborn as sns

In [2]:
CHUNKSIZE = 100000

TRAIN_PATH        = "../train_StationPathInfo.csv"
TRAIN_PATH_EX     = "../train_StationPathInfoEx.csv"
TEST_PATH         = "../test_StationPathInfo.csv"
TEST_PATH_EX      = "../test_StationPathInfoEx.csv"

TRAIN_DATE        = "../train_date.csv"
TRAIN_NUMERIC     = "../train_numeric.csv"
TRAIN_CATEGORICAL = "../train_categorical.csv"
TEST_DATE         = "../test_date.csv"
TEST_NUMERIC      = "../test_numeric.csv"
TEST_CATEGORICAL  = "../test_categorical.csv"

SEED = 0
CHUNKSIZE = 50000
NROWS = 1200000

ID_COLUMN = 'Id'
TARGET_COLUMN = 'Response'
    
    

In [3]:
chunk_train_num = pd.read_csv(TRAIN_NUMERIC, nrows=NROWS, chunksize=CHUNKSIZE)
chunk_test_num  = pd.read_csv(TEST_NUMERIC, nrows=NROWS, chunksize=CHUNKSIZE)
chunk_train_cat = pd.read_csv(TRAIN_CATEGORICAL, nrows=NROWS, chunksize=CHUNKSIZE)
chunk_test_cat  = pd.read_csv(TEST_CATEGORICAL, nrows=NROWS, chunksize=CHUNKSIZE)

df_train_num = chunk_train_num.get_chunk(10)
df_test_num  = chunk_test_num.get_chunk(10)
df_train_cat = chunk_train_cat.get_chunk(10)
df_test_cat  = chunk_test_cat.get_chunk(10)

df_train_num.head()

,Id,L0_S0_F0,L0_S0_F2,L0_S0_F4,L0_S0_F6,L0_S0_F8,L0_S0_F10,L0_S0_F12,L0_S0_F14,L0_S0_F16,...,L3_S50_F4245,L3_S50_F4247,L3_S50_F4249,L3_S50_F4251,L3_S50_F4253,L3_S51_F4256,L3_S51_F4258,L3_S51_F4260,L3_S51_F4262,Response
0,4,0.030,-0.034,-0.197,-0.179,0.118,0.116,-0.015,-0.032,0.020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,7,0.088,0.086,0.003,-0.052,0.161,0.025,-0.015,-0.072,-0.225,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,9,-0.036,-0.064,0.294,0.330,0.074,0.161,0.022,0.128,-0.026,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,11,-0.055,-0.086,0.294,0.330,0.118,0.025,0.030,0.168,-0.169,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


### 通過ステーション情報・時刻情報を読み出す。

In [4]:
# 読み出す。

df_train_bin = pd.read_csv(TRAIN_PATH)
df_test_bin  = pd.read_csv(TEST_PATH)
df_pass = pd.concat([df_train_bin, df_test_bin])

In [5]:
# 時系列でソートしたヒートマップを出してみる。

TEST_START_TIME      = "../test_StartEndTime.csv"
TRAIN_START_TIME     = "../train_StartEndTime.csv"

df_start_train = pd.read_csv(TRAIN_START_TIME)
df_start_train = df_start_train.ix[:,['Id','StartTime','EndTime', 'Response']]
df_start_test = pd.read_csv(TEST_START_TIME)
df_start_test['Response'] = -1
df_time = pd.concat([df_start_train, df_start_test])


C:\Users\Tomonobu\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  import sys


## S32 の numeric, categorical とマージし、S32に限定して予想してみる。

In [13]:
df_station32_train_cat = pd.read_csv("./train_categorical_station_32.csv")
df_station32_test_cat  = pd.read_csv("./test_categorical_station_32.csv")
df_station32_cat = pd.concat([df_station32_train_cat, df_station32_test_cat]).fillna('T0')


df_station32_train_num = pd.read_csv("./train_numeric_station_32.csv")
df_station32_test_num  = pd.read_csv("./test_numeric_station_32.csv")
df_station32_num = pd.concat([df_station32_train_num, df_station32_test_num])


C:\Users\Tomonobu\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [48]:
df_analyze = pd.merge(df_pass, df_time, on='Id', how='left')
df_analyze = pd.merge(df_analyze, df_station32_cat, on='Id', how='left')
df_analyze = pd.merge(df_analyze, df_station32_num, on='Id', how='left')
df_analyze = df_analyze[df_analyze['L3_S32_D3852'] == 1]
df_analyze = df_analyze.drop(['Id'], axis=1)

In [49]:
df_analyze.head(5)

,L0_S0_D1,L0_S1_D26,L0_S2_D34,L0_S3_D70,L0_S4_D106,L0_S5_D115,L0_S6_D120,L0_S7_D137,L0_S8_D145,L0_S9_D152,...,L3_S49_D4208,L3_S50_D4242,L3_S51_D4255,StartTime,EndTime,Response,L3_S32_F3851,L3_S32_F3853,L3_S32_F3854,L3_S32_F3850
55,0,0,0,0,0,0,0,0,0,0,...,0,0,0,555.38,556.25,0,T1,T0,T16,0.044
71,1,1,0,1,0,1,0,1,1,1,...,0,0,0,1164.57,1170.89,0,T1,T0,T16,-0.055
80,0,0,0,0,0,0,0,0,0,0,...,0,0,0,456.80,493.38,0,T1,T0,T256,0.009
141,1,1,1,0,0,1,0,1,1,0,...,0,0,0,1339.10,1342.73,0,T0,T0,T0,0.006
321,1,1,1,0,0,1,1,0,1,1,...,0,0,0,1575.48,1578.18,0,T1,T0,T128,-0.019


In [50]:

def is_bit_on(s, pos):
    if type(s) is str:
        r = parse("T{}", s)
        val = (int(r[0]))
        mask = 1 << pos
        if val & mask:
            return 1
        else:
            return 0
    else:
        return 0

In [51]:
def decode_categorical_data(df, column_list):
    for column_name, item in df.iteritems():
        if column_name in column_list:
            vallist = item.unique()
            bitlist = []
            for c in vallist:
                if type(c) is str:
                    r = parse("T{}", c)
                    val = int(r[0])
                    if val < 0:
                        val = val + 4294967296
                    if 0 < val:
                        bitpos = int(np.log2(val))
                        bitlist.append(bitpos)

            for bit in bitlist:
                df[column_name + "_bit_" + str(bit)] = df[column_name].apply(lambda x:is_bit_on(x, bit)).astype('int8')
            df.drop(column_name, axis=1, inplace=True)


In [52]:
decode_categorical_data(df_analyze, ['L3_S32_F3851', 'L3_S32_F3853', 'L3_S32_F3854'])


In [53]:
df_analyze.head(5)

,L0_S0_D1,L0_S1_D26,L0_S2_D34,L0_S3_D70,L0_S4_D106,L0_S5_D115,L0_S6_D120,L0_S7_D137,L0_S8_D145,L0_S9_D152,...,L3_S32_F3854_bit_3,L3_S32_F3854_bit_2,L3_S32_F3854_bit_31,L3_S32_F3854_bit_0,L3_S32_F3854_bit_10,L3_S32_F3854_bit_5,L3_S32_F3854_bit_15,L3_S32_F3854_bit_6,L3_S32_F3854_bit_16,L3_S32_F3854_bit_14
55,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
71,1,1,0,1,0,1,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
80,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
141,1,1,1,0,0,1,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
321,1,1,1,0,0,1,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0


In [75]:
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, log_loss, make_scorer

def calc_mcc(cf_mat):
    tn, fp, fn, tp = cf_mat.ravel()
    print(tn, fp, fn, tp)
    mcc = (tp * tn - fp * fn) / np.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))
    return mcc

def mcc_scorer(y_true, y_pred):
    cf_mat = confusion_matrix(y_true, y_pred)
    return calc_mcc(cf_mat)

def separate_X_y(df):
    X = df.drop(['Response'], axis=1)
    y = df['Response']
    return X, y

def train_with_r_forest(df):
    X_train, y_train = separate_X_y(df)

    cl_weight = {0:0.3, 1:0.7}
    
    # Grid Search
    params={'max_depth': [5, 10, 20],
            'subsample': [0.95],
            'colsample_bytree': [1.0],
            'scale_pos_weight': [2, 3]
    }

    xgb_model = xgb.XGBClassifier()
    gs = GridSearchCV(xgb_model,
                      params,
                      cv=5,
                      scoring={'mcc' : make_scorer(mcc_scorer)},
                      n_jobs=1,
                      verbose=2,
                      refit='mcc')

    gs.fit(X_train, y_train)
#rf = RandomForestClassifier(max_depth=10,n_estimators=50, random_state=33)
#    rf.fit(X_train, y_train)
    return gs


In [76]:

# アンダーサンプリングはしない。

def train_rf_parameter(df):
    df = df[(df['Response'] == 0) | (df['Response'] == 1)]
    df_train, df_test = train_test_split(df, random_state=33)
    
    df_train_ok   = df_train[df_train['Response'] == 0]
    df_train_ng   = df_train[df_train['Response'] == 1]
#    undersample_rate = len(df_train_ng) * 30 / len(df_train_ok)
    undersample_rate = 1
    df_train_ok_sample = df_train_ok.sample(frac = undersample_rate)
    df_train = pd.concat([df_train_ok_sample, df_train_ng])

    rf = train_with_r_forest(df_train)
    X_test, y_test = separate_X_y(df_test)
    y_pred = rf.predict(X_test)
    print(y_pred)   
    
    cf_mat = confusion_matrix(y_test, y_pred)
    print(cf_mat)

    mcc = calc_mcc(cf_mat)
    print(mcc)    
    return rf

model = train_rf_parameter(df_analyze)


Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV] colsample_bytree=1.0, max_depth=5, scale_pos_weight=2, subsample=0.95 
3434 82 98 68
13817 245 280 383
[CV]  colsample_bytree=1.0, max_depth=5, scale_pos_weight=2, subsample=0.95, total=   1.6s
[CV] colsample_bytree=1.0, max_depth=5, scale_pos_weight=2, subsample=0.95 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.7s remaining:    0.0s


3422 94 93 73
13791 271 267 396
[CV]  colsample_bytree=1.0, max_depth=5, scale_pos_weight=2, subsample=0.95, total=   1.6s
[CV] colsample_bytree=1.0, max_depth=5, scale_pos_weight=2, subsample=0.95 
3445 71 104 62
13802 260 276 387
[CV]  colsample_bytree=1.0, max_depth=5, scale_pos_weight=2, subsample=0.95, total=   1.6s
[CV] colsample_bytree=1.0, max_depth=5, scale_pos_weight=2, subsample=0.95 
3421 94 79 87
13791 272 281 382
[CV]  colsample_bytree=1.0, max_depth=5, scale_pos_weight=2, subsample=0.95, total=   1.6s
[CV] colsample_bytree=1.0, max_depth=5, scale_pos_weight=2, subsample=0.95 
3428 87 91 74
13791 272 275 389
[CV]  colsample_bytree=1.0, max_depth=5, scale_pos_weight=2, subsample=0.95, total=   1.5s
[CV] colsample_bytree=1.0, max_depth=5, scale_pos_weight=3, subsample=0.95 
3392 124 66 100
13650 412 189 474
[CV]  colsample_bytree=1.0, max_depth=5, scale_pos_weight=3, subsample=0.95, total=   1.5s
[CV] colsample_bytree=1.0, max_depth=5, scale_pos_weight=3, subsample=0.95 
33

[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:  1.9min finished


[0 0 0 ..., 0 0 0]
[[5679  180]
 [ 136  141]]
5679 180 136 141
0.445992585945


In [77]:
model.best_estimator_.feature_importances_

array([ 0.0014985 ,  0.        ,  0.00999001,  0.01298701,  0.01298701,
        0.00849151,  0.01748252,  0.002997  ,  0.000999  ,  0.02047952,
        0.00999001,  0.01198801,  0.001998  ,  0.        ,  0.00749251,
        0.002997  ,  0.0044955 ,  0.01598402,  0.00899101,  0.00599401,
        0.        ,  0.00649351,  0.0034965 ,  0.00799201,  0.0044955 ,
        0.0044955 ,  0.01748252,  0.01598402,  0.0014985 ,  0.        ,
        0.        ,  0.000999  ,  0.        ,  0.02697303,  0.02647353,
        0.01448551,  0.00949051,  0.        ,  0.03096903,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.18781219,  0.16783217,  0.16133866,
        0.00549451,  0.        ,  0.01998002,  0.00499501,  0.00899101,
        0.00899101,  0.02897103,  0.01398601,  0.01498501,  0.01598402,
        0.00999001,  0.0024975 ,  0.01698302,  0.0004995 ,  0.00

In [78]:
df_feature_importance = pd.DataFrame(columns=['name', 'importance'])

In [79]:
df_feature_importance.loc[:, 'importance'] = model.best_estimator_.feature_importances_
df_feature_importance.loc[:, 'name'] = np.array(df_analyze.drop(['Response'], axis=1).columns)
df_feature_importance.sort_values('importance', ascending=False)

,name,importance
52,StartTime,0.187812
53,EndTime,0.167832
54,L3_S32_F3850,0.161339
38,L3_S38_D3953,0.030969
61,L3_S32_F3854_bit_1,0.028971
33,L3_S33_D3856,0.026973
34,L3_S34_D3875,0.026474
9,L0_S9_D152,0.020480
57,L3_S32_F3854_bit_4,0.019980
6,L0_S6_D120,0.017483


In [69]:
np.array(df_analyze.columns)

array(['L0_S0_D1', 'L0_S1_D26', 'L0_S2_D34', 'L0_S3_D70', 'L0_S4_D106',
       'L0_S5_D115', 'L0_S6_D120', 'L0_S7_D137', 'L0_S8_D145',
       'L0_S9_D152', 'L0_S10_D216', 'L0_S11_D280', 'L0_S12_D331',
       'L0_S13_D355', 'L0_S14_D360', 'L0_S15_D395', 'L0_S16_D423',
       'L0_S17_D432', 'L0_S18_D437', 'L0_S19_D454', 'L0_S20_D462',
       'L0_S21_D469', 'L0_S22_D543', 'L0_S23_D617', 'L1_S24_D677',
       'L1_S25_D1854', 'L2_S26_D3037', 'L2_S27_D3130', 'L2_S28_D3223',
       'L3_S29_D3316', 'L3_S30_D3496', 'L3_S31_D3836', 'L3_S32_D3852',
       'L3_S33_D3856', 'L3_S34_D3875', 'L3_S35_D3886', 'L3_S36_D3919',
       'L3_S37_D3942', 'L3_S38_D3953', 'L3_S39_D3966', 'L3_S40_D3981',
       'L3_S41_D3997', 'L3_S42_D4029', 'L3_S43_D4062', 'L3_S44_D4101',
       'L3_S45_D4125', 'L3_S46_D4135', 'L3_S47_D4140', 'L3_S48_D4194',
       'L3_S49_D4208', 'L3_S50_D4242', 'L3_S51_D4255', 'StartTime',
       'EndTime', 'Response', 'L3_S32_F3850', 'L3_S32_F3851_bit_0',
       'L3_S32_F3853_bit_0', 'L3_S32

In [ ]:
def train_and_predict_one_chnuk(df_train):
    df_train_ok   = df_train[df_train['Response'] == 0]
    df_train_ng   = df_train[df_train['Response'] == 1]
    df_test = df_train[df_train['Response'] == -1]
    
    undersample_rate = len(df_train_ng) * 20 / len(df_train_ok)
    df_train_ok_sample = df_train_ok.sample(frac = undersample_rate)

    df_train_balance = pd.concat([df_train_ok_sample, df_train_ng])
    
    rf = train_with_r_forest(df_train_balance)

    X_test, y_test = separate_X_y(df_test)
    y_pred = rf.predict(X_test)
      
    df_result_add = pd.DataFrame(columns=['Id', 'Response'])
    df_result_add.loc[:, 'Id']       = df_test['Id'].values
    df_result_add.loc[:, 'Response'] = y_pred
    
    return df_result_add

df_result = train_and_predict_one_chnuk(df_train)

In [112]:
df_pass.columns

Index(['Id', 'L0_S0_D1', 'L0_S1_D26', 'L0_S2_D34', 'L0_S3_D70', 'L0_S4_D106',
       'L0_S5_D115', 'L0_S6_D120', 'L0_S7_D137', 'L0_S8_D145', 'L0_S9_D152',
       'L0_S10_D216', 'L0_S11_D280', 'L0_S12_D331', 'L0_S13_D355',
       'L0_S14_D360', 'L0_S15_D395', 'L0_S16_D423', 'L0_S17_D432',
       'L0_S18_D437', 'L0_S19_D454', 'L0_S20_D462', 'L0_S21_D469',
       'L0_S22_D543', 'L0_S23_D617', 'L1_S24_D677', 'L1_S25_D1854',
       'L2_S26_D3037', 'L2_S27_D3130', 'L2_S28_D3223', 'L3_S29_D3316',
       'L3_S30_D3496', 'L3_S31_D3836', 'L3_S32_D3852', 'L3_S33_D3856',
       'L3_S34_D3875', 'L3_S35_D3886', 'L3_S36_D3919', 'L3_S37_D3942',
       'L3_S38_D3953', 'L3_S39_D3966', 'L3_S40_D3981', 'L3_S41_D3997',
       'L3_S42_D4029', 'L3_S43_D4062', 'L3_S44_D4101', 'L3_S45_D4125',
       'L3_S46_D4135', 'L3_S47_D4140', 'L3_S48_D4194', 'L3_S49_D4208',
       'L3_S50_D4242', 'L3_S51_D4255'],
      dtype='object')

In [30]:
df_cat_train = pd.read_csv(TEST_CATEGORICAL,  
                            usecols=use_cat_columns, nrows=NROWS)
df_cat_test  = pd.read_csv(TEST_CATEGORICAL,  
                            usecols=use_cat_columns, nrows=NROWS)

C:\Users\Tomonobu\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (1773,1774,1775,1776,1777,1778,1779,1780,1781,1782,1783,1784,1785,1786,1787,1788,1789,1790,1791,1792,1793,1794,1795,1796,1797,1798,1799,1800,1801,1802,1803,1804,1805,1806,1807,1808,1809,1810,1811,1812,1813,1814,1815,1816,1817,1818,1819,1820,1821,1822,1823,1824,1825,1826,1827,1828,1829,1830,1831,1832,1833,1834,1835,1836,1837,1838,1839,1840,1841,1842,1843,1844,1845,1846,1847,1848,1849,1850,1851,1852,1853,1854,1855,1856,1857,1858,1859,1860,1861,1862,1863,1864,1865,1866,1867,1868,1869,1870,1871,1872,1873,1874,1875,1876,1877,1878,1879,1880,1881,1882,1883,1884,1885,1886,1887,1888,1889,1890,1891,1892,1893,1894,1895,1896,1897,1898,1899,1900,1901,1902,1903,1904,1905,1906,1907,1908,1909,1910,1911,1912,1913,1914,1915,1916,1917,1918,1919,1920,1921,1922,1923,1924,1925,1926,1927,1928,1929,1930,1931,1932,1933,1934,1935,1936,1937,1938,1939,1940,1941,1942,1943,1944,1945,1946,1947,1948,1949,1950,19

In [31]:
df_num_train = pd.read_csv(TEST_NUMERIC,  
                            usecols=use_num_columns, nrows=NROWS)
df_num_test  = pd.read_csv(TEST_NUMERIC,  
                            usecols=use_num_columns, nrows=NROWS)

In [29]:
use_station_list = [29, 30, 31, 33]

use_cat_columns = ['Id']
use_num_columns = ['Id']

for i in range(STATION_NUM):
    if i in use_station_list:
        use_cat_columns += cat_feature_list[i]
        use_num_columns += num_feature_list[i]

print(use_cat_columns)
print(use_num_columns)
print(len(use_cat_columns))
print(len(use_num_columns))

['Id', 'L3_S29_F3317', 'L3_S29_F3320', 'L3_S29_F3323', 'L3_S29_F3326', 'L3_S29_F3329', 'L3_S29_F3332', 'L3_S29_F3335', 'L3_S29_F3338', 'L3_S29_F3341', 'L3_S29_F3344', 'L3_S29_F3347', 'L3_S29_F3350', 'L3_S29_F3353', 'L3_S29_F3356', 'L3_S29_F3359', 'L3_S29_F3362', 'L3_S29_F3364', 'L3_S29_F3366', 'L3_S29_F3369', 'L3_S29_F3372', 'L3_S29_F3375', 'L3_S29_F3378', 'L3_S29_F3381', 'L3_S29_F3384', 'L3_S29_F3387', 'L3_S29_F3390', 'L3_S29_F3392', 'L3_S29_F3394', 'L3_S29_F3397', 'L3_S29_F3400', 'L3_S29_F3403', 'L3_S29_F3406', 'L3_S29_F3409', 'L3_S29_F3411', 'L3_S29_F3414', 'L3_S29_F3416', 'L3_S29_F3418', 'L3_S29_F3420', 'L3_S29_F3423', 'L3_S29_F3426', 'L3_S29_F3429', 'L3_S29_F3432', 'L3_S29_F3435', 'L3_S29_F3438', 'L3_S29_F3441', 'L3_S29_F3444', 'L3_S29_F3446', 'L3_S29_F3448', 'L3_S29_F3451', 'L3_S29_F3454', 'L3_S29_F3457', 'L3_S29_F3460', 'L3_S29_F3463', 'L3_S29_F3466', 'L3_S29_F3469', 'L3_S29_F3472', 'L3_S29_F3475', 'L3_S29_F3478', 'L3_S29_F3481', 'L3_S29_F3484', 'L3_S29_F3487', 'L3_S29_F3490', '

In [34]:
df_start_test.head()
df_start = pd.concat([df_start_train, df_start_test])
df_start.head()

,Id,StartTime,EndTime,Response
0,4,82.24,87.29,0
1,6,1313.12,1315.75,0
2,7,1618.70,1624.42,0
3,9,1149.20,1154.16,0
4,11,602.64,606.02,0


In [35]:
df_all = pd.merge(df_start, df_pass, on='Id',  how='left')
df_all = df_all.fillna(-1500)

In [36]:
df_all = pd.merge(df_all, df_num_train, on='Id', how='left')
df_all = pd.merge(df_all, df_num_test,  on='Id', how='left')


In [38]:
df_all = df_all.fillna(-1500)
df_all.head

<bound method NDFrame.head of               Id  StartTime  EndTime  Response  L0_S0_D1  L0_S1_D26  \
0              4      82.24    87.29         0         1          1   
1              6    1313.12  1315.75         0         0          0   
2              7    1618.70  1624.42         0         1          1   
3              9    1149.20  1154.16         0         1          1   
4             11     602.64   606.02         0         1          1   
5             13    1331.66  1339.73         0         1          1   
6             14    1662.63  1664.04         0         0          0   
7             16     791.22   804.36         0         0          0   
8             18     517.64   518.08         0         1          1   
9             23     156.27   157.89         0         0          0   
10            26    1104.78  1105.95         0         1          1   
11            27     392.85   401.41         0         1          1   
12            28      55.44    62.10         0 

In [39]:
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, log_loss, make_scorer

def calc_mcc(cf_mat):
    tn, fp, fn, tp = cf_mat.ravel()
    print(tn, fp, fn, tp)
    mcc = (tp * tn - fp * fn) / np.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))
    return mcc

def mcc_scorer(y_true, y_pred):
    cf_mat = confusion_matrix(y_true, y_pred)
    return calc_mcc(cf_mat)

def separate_X_y(df):
    X = df.drop(['Response'], axis=1)
    y = df['Response']
    return X, y

def train_with_r_forest(df):
    X_train, y_train = separate_X_y(df)

    cl_weight = {0:0.3, 1:0.7}
    
    # Grid Search
    params={'max_depth': [25],
            'subsample': [0.95],
            'colsample_bytree': [1.0],
            'scale_pos_weight': [2]
    }

    xgb_model = xgb.XGBClassifier()
    gs = GridSearchCV(xgb_model,
                      params,
                      cv=5,
                      scoring={'mcc' : make_scorer(mcc_scorer)},
                      n_jobs=1,
                      verbose=2,
                      refit='mcc')

    gs.fit(X_train, y_train)
#rf = RandomForestClassifier(max_depth=10,n_estimators=50, random_state=33)
#    rf.fit(X_train, y_train)
    return gs


C:\Users\Tomonobu\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [41]:
def train_rf_parameter(df):
    df = df[(df['Response'] == 0) | (df['Response'] == 1)]
    df_train, df_test = train_test_split(df, random_state=33)
    
    df_train_ok   = df_train[df_train['Response'] == 0]
    df_train_ng   = df_train[df_train['Response'] == 1]
    undersample_rate = len(df_train_ng) * 30 / len(df_train_ok)
    df_train_ok_sample = df_train_ok.sample(frac = undersample_rate)
    df_train = pd.concat([df_train_ok_sample, df_train_ng])

    rf = train_with_r_forest(df_train)
    X_test, y_test = separate_X_y(df_test)
    y_pred = rf.predict(X_test)
    print(y_pred)   
    
    cf_mat = confusion_matrix(y_test, y_pred)
    print(cf_mat)

    mcc = calc_mcc(cf_mat)
    print(mcc)    
    return rf

model = train_rf_parameter(df_all)


Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] colsample_bytree=1.0, max_depth=25, scale_pos_weight=2, subsample=0.95 
31041 141 852 188
124717 11 572 3585
[CV]  colsample_bytree=1.0, max_depth=25, scale_pos_weight=2, subsample=0.95, total= 4.5min
[CV] colsample_bytree=1.0, max_depth=25, scale_pos_weight=2, subsample=0.95 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  4.6min remaining:    0.0s


31060 122 859 181
124720 8 621 3536
[CV]  colsample_bytree=1.0, max_depth=25, scale_pos_weight=2, subsample=0.95, total= 4.6min
[CV] colsample_bytree=1.0, max_depth=25, scale_pos_weight=2, subsample=0.95 
31055 127 875 164
124718 10 676 3482
[CV]  colsample_bytree=1.0, max_depth=25, scale_pos_weight=2, subsample=0.95, total= 4.5min
[CV] colsample_bytree=1.0, max_depth=25, scale_pos_weight=2, subsample=0.95 
31061 121 859 180
124714 14 520 3638
[CV]  colsample_bytree=1.0, max_depth=25, scale_pos_weight=2, subsample=0.95, total= 4.2min
[CV] colsample_bytree=1.0, max_depth=25, scale_pos_weight=2, subsample=0.95 
31051 131 849 190
124718 10 660 3498
[CV]  colsample_bytree=1.0, max_depth=25, scale_pos_weight=2, subsample=0.95, total= 4.3min


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed: 22.4min finished


[0 0 0 ..., 0 0 0]
[[293022   1233]
 [  1371    311]]
293022 1233 1371 311
0.18856737844
